In [14]:
# Import boto3 and connect to S3
import boto3
s3 = boto3.resource('s3')

In [16]:
# Upload the file "welcome.txt" to the bucket "aigc-bj-team1"
import os
bucket_name = 'aigc-bj-team1'
s3path = 'https://aigc-bj-team1' + '.s3.amazonaws.com/'
audiopath = []
mp3files = [file for file in os.listdir('audiofile') if file.endswith('.wav')]
for file in mp3files:
    s3.Bucket(bucket_name).upload_file('audiofile/' + file, file)
    
# Generate a pre-signed URL for the objecct "welcome.txt", valid for 1 hour
    url = s3.meta.client.generate_presigned_url(
        ClientMethod='get_object',
        Params={
            'Bucket': bucket_name,
            'Key': file
        },
        ExpiresIn=3600
    )

# Print out the URL
    audiopath.append(s3path+file)
audiopath

['https://aigc-bj-team1.s3.amazonaws.com/Recording.wav']

In [17]:
import time
import boto3
import urllib.request
import json
def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='wav',
        # Set multiple language codes, for english and chinese
        IdentifyLanguage = True,
        LanguageOptions=['en-US', 'zh-CN']
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                return job['TranscriptionJob']['Transcript']['TranscriptFileUri']
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)
    return None

def encoder_url(url):
    response = urllib.request.urlopen(url)
    data = response.read().decode('utf-8')
    jsondata = json.loads(data)
    return(jsondata['results']['transcripts'][0]['transcript'])

def main():
    transcribe_client = boto3.client('transcribe')
    file_uri = 'https://aigc-bj-team1.s3.amazonaws.com/Recording.wav'
    
    timestamp = str(time.time()).split('.')[0]
    url2 = transcribe_file('Example-Transcribe-job-{}'.format(timestamp), file_uri, transcribe_client)
    a = encoder_url(url2)
    print(a)
if __name__ == '__main__':
    main()
    


Waiting for Example-Transcribe-job-1692336329. Current status is IN_PROGRESS.
Waiting for Example-Transcribe-job-1692336329. Current status is IN_PROGRESS.
Waiting for Example-Transcribe-job-1692336329. Current status is IN_PROGRESS.
Waiting for Example-Transcribe-job-1692336329. Current status is IN_PROGRESS.
Job Example-Transcribe-job-1692336329 is COMPLETED.
今天天津、上海，现在是八月十八日二零二三年。


In [11]:
import gradio as gr
import boto3
import time
import json

bucket_name = 'aigc-bj-team1'


def upload_to_s3(file_path, bucket):
    object_name = "example.mp3" # unique in a bucket
    s3_client = boto3.client('s3')
    try:
        s3_client.upload_file(file_path, bucket, object_name)
    except:
        return False
    return "s3://"+bucket+"/"+object_name

def get_transcript(bucket, key):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    response_json = json.loads(obj['Body'].read())
    transcript = response_json['results']['transcripts'][0]['transcript']
    return transcript

def transcribe(audio):
    bucket = bucket_name
    s3_url = upload_to_s3(audio, bucket)
    # create a transciption job
    job_name = "ExampleJob" + str(time.time()) # a unique name in your Amazon Transcribe
    transcribe_client = boto3.client('transcribe')
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': s3_url},
        MediaFormat='mp3',
        IdentifyLanguage = True,
        LanguageOptions=['en-US', 'zh-CN'],
        OutputBucketName=bucket,
        OutputKey='example_text.txt'
    )
    max_tries = 100
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                transcript = get_transcript(bucket,'example_text.txt')
                return transcript
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)
    return "Fail"

gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="upload", format="mp3", type="filepath"),
    outputs="text").launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://0e9a0a5dca978044c4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Waiting for ExampleJob1692337999.4910924. Current status is IN_PROGRESS.
Waiting for ExampleJob1692337999.4910924. Current status is IN_PROGRESS.
Job ExampleJob1692337999.4910924 is COMPLETED.
